# Wikipedia爬蟲練習
## 範例：練習是從Wikipedia中爬取文章。先定義一個搜尋的關鍵字，擷取該關鍵字詞的文章。

In [1]:
import requests
import re
from bs4 import BeautifulSoup

### 先定義一個我們想搜尋的字詞，並將它轉換成UTF-8編碼後的URL

In [2]:
input_keyword = "投資"  # 這裡可以自己定義有興趣的關鍵字

utf8_url = repr(input_keyword.encode('UTF-8')).upper()  # 編碼成UTF-8並轉成大寫字元
utf8_url = utf8_url.replace("\\X", "%")                 # 用 '%' 取代 '\X' 
print("%s: %s" % (input_keyword, utf8_url[2:-1:1]))     # 擷取中間的編碼結果

# 組成Wiki關鍵字搜尋的網址格式
root_keyword_link = '/wiki/' + utf8_url[2:-1:1]
print(root_keyword_link)

投資: %E6%8A%95%E8%B3%87
/wiki/%E6%8A%95%E8%B3%87


### 範例1：送出關鍵字請求後，爬取該關鍵字的文章內容

In [3]:
# 模擬封包的標頭
headers = {
    'authority': 'zh.wikipedia.org',
    'method': 'GET',
    'path': '/wiki/' + root_keyword_link,
    'scheme': 'https',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7,zh-CN;q=0.6',
    'cookie': 'GeoIP=TW:TPE:Taipei:25.05:121.53:v4; TBLkisOn=0; mwPhp7Seed=8b8; WMF-Last-Access-Global=04-Jun-2019; WMF-Last-Access=04-Jun-2019',
    'dnt': '1',
    #'if-modified-since': 'Tue, 04 Jun 2019 12:03:22 GMT',
    'referer': 'https://zh.wikipedia.org/wiki/Wikipedia:%E9%A6%96%E9%A1%B5',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36'
}    

url = 'https://zh.wikipedia.org' + root_keyword_link  # 組合關鍵字查詢URL
resp = requests.get(url, headers=headers)
resp.encoding = 'utf-8'

html = BeautifulSoup(resp.text, "lxml")
content = html.find(name='div', attrs={'id':'mw-content-text'}).find_all(name='p')

#
# 解析回傳資料，並萃取文章內容
#
for paragraph in content:
    print(paragraph.get_text())


投資在財務（金融）及經濟方面，各有不同的意義。財務投資是透過完善的分析，對於本金、報酬可達一定程度的預估，將資金投入那些預期有所增長的標的上[1]。而完全對立的作法，像未經分析、且在本金及報酬上也沒有把握，亦不考慮風險就投入資金者，就是投機、或賭博。因此，有些並未經過完善分析，即買進股權持有者，例如買進共同基金的人，也可以稱為投機者。事實上，根據效率市場假說，即使針對股市資料來完整分析者，也不算是理性。依此定義延伸下去，所有自以為理性的股權持有者，也不能視為投資者，而是投機者。

投資與儲蓄、或遞延消費，具有一定的相關性。投資牽涉到許多經濟領域的活動，例如企（事）業管理、及財務金融等，無論是為家計、事業、或政府。

投資、投機及賭博的不同：

在經濟學、或總體經濟方面，投資是指每單位時間內，所購入資本（財）的金額。所謂的資本，就是目的不在於消費、而主要著眼於其未來的生產能力，如公共建設、鐵路、工廠等。而人力資本方面的投入，包括新設立學校、或在職進修機構等。存貨（營運資本）的投資，就是指各種事業為營運需要、購置存貨的行為，此一行為可能是經過審慎的管理流程、也有可能只是因應需要而為之；而進貨的存量有時為正、有時候也可能變成負值。

在衡量政府收支的GDP計算上， 如GDP = C + I + G + NX 中，「I」就是投資（Investment，又稱：固定資本毛額）是國內生產總值（GDP）的一部分。投資與收入和利率的關聯〔I = (Y，i)〕, 可理解為，收入的增加促進更高額的投資，但是，高利率阻礙投資（因為借錢的費用變得昂貴）。

投資是在確認可獲得一定擔保（如存款保險、或事業的投入資本）後，將資金存放、或投入於具有安全性的組織（如金融機構、上市公司）的一種承諾。而與前述投資意義明顯對立的一些行為，如定期定額（dollor cost averaging）、波段操作（market timing）、分散押寶（diversification）等，則與投機比較接近。

投資工具：[2]



### 範例2：從爬取的文章內容中，擷取出有外部連結的關鍵字。這些關鍵字在文章中是以藍色字體顯示，會連到外部的網頁，並解釋其內容。

In [4]:
for ext_link in content:
    a_tag = ext_link.find_all('a', href=re.compile("^(/wiki/)((?!;)\S)*$"))
    if len(a_tag) > 0:
        for link_string in a_tag:
            a_link = link_string["href"]       # 外部連結的網址
            a_keyword = link_string.get_text()  # 外部連結的中文名稱
            print("外部連結: [%s] %s" % (a_keyword, a_link))

外部連結: [投機] /wiki/%E6%8A%95%E6%9C%BA
外部連結: [賭博] /wiki/%E8%B5%8C%E5%8D%9A
外部連結: [效率市場假說] /wiki/%E6%95%88%E7%8E%87%E5%B8%82%E5%A0%B4%E5%81%87%E8%AA%AA
外部連結: [理性] /wiki/%E7%90%86%E6%80%A7
外部連結: [股權] /wiki/%E8%82%A1%E6%9D%B1%E6%AC%8A%E7%9B%8A
外部連結: [儲蓄] /wiki/%E5%82%A8%E8%93%84
外部連結: [經濟學] /wiki/%E7%BB%8F%E6%B5%8E%E5%AD%A6
外部連結: [資本] /wiki/%E8%B5%84%E6%9C%AC
外部連結: [國內生產總值] /wiki/%E5%9B%BD%E5%86%85%E7%94%9F%E4%BA%A7%E6%80%BB%E5%80%BC
外部連結: [存款保險] /wiki/%E5%AD%98%E6%AC%BE%E4%BF%9D%E9%9A%AA


## 作業：接下來定義一個爬蟲函數，這個函數的主要工作為：
### (1) 爬取當前關鍵字的解釋，並存入檔案(因為文章內容太多會佔滿整個頁面，所以存程檔案，方便後續檢視)
### (2) 萃取出當前關鍵字所引用的外部連結，當作新的查詢關鍵字
### (3) 把第(2)擷取到的關鍵字當作新的關鍵字，回到第(1)步，爬取新的關鍵字解釋。

In [5]:
def WikiArticle(key_word_link, key_word, recursive):
    
    if (recursive <= max_recursive_depth):
        print("遞迴層[%d] - %s (%s)" % (recursive, key_word_link, key_word))
        
        # 模擬封包的標頭
        headers = {
            'authority': 'zh.wikipedia.org',
            'method': 'GET',
            'path': '/wiki/' + key_word_link,
            'scheme': 'https',
            'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3',
            'accept-encoding': 'gzip, deflate, br',
            'accept-language': 'zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7,zh-CN;q=0.6',
            'cookie': 'GeoIP=TW:TPE:Taipei:25.05:121.53:v4; TBLkisOn=0; mwPhp7Seed=8b8; WMF-Last-Access-Global=04-Jun-2019; WMF-Last-Access=04-Jun-2019',
            'dnt': '1',
            #'if-modified-since': 'Tue, 04 Jun 2019 12:03:22 GMT',
            'referer': 'https://zh.wikipedia.org/wiki/Wikipedia:%E9%A6%96%E9%A1%B5',
            'upgrade-insecure-requests': '1',
            'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36'
        }    

        url = 'https://zh.wikipedia.org' + key_word_link  # 組合關鍵字查詢URL
        resp = requests.get(url, headers=headers)
        resp.encoding = 'utf-8'

        html = BeautifulSoup(resp.text, "lxml")
        content = html.find(name='div', attrs={'id':'mw-content-text'}).find_all(name='p')
        
        #
        # Part 1: 請參考範例1，爬取當前關鍵字的文章內容。
        #         因為內容太多，我們把它寫入檔案，並以關鍵字作為檔案名稱，以便稍後查閱內容。
        #         請先建立一個名為"WikiArticle"的資料夾，爬取到的文章內容會放在這個資料夾底下。
        #
        import os
        
        output_dir = 'WikiArticle'
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)

        
        #
        # Part 2: 請參考範例2，萃取出本篇文章中所延伸引用的外部連結，並儲存在external_link_dict
        #
        external_link_dict = dict({})
        for ext_link in content:
            a_tag = ext_link.find_all('a', href=re.compile("^(/wiki/)((?!;|#)\S)*$"))
            if len(a_tag) > 0:
                for link_string in a_tag:
                    a_link = link_string["href"]       # 外部連結的網址
                    a_keyword = link_string.get_text()  # 外部連結的中文名稱
                    external_link_dict[a_link] = a_keyword
        
        #
        # Part 3: 將Part 2所收集的外部連結，當作新的關鍵字，繼續迭代深入爬蟲
        #
        if (len(external_link_dict) > 0):
            
            recursive = recursive + 1  # 遞迴深度加1
            
            for k, v in external_link_dict.items():
                WikiArticle(k, v, recursive)  # 再次呼叫同樣的函數，執行同樣的流程
                

### 執行前個步驟定義好的爬蟲主程式

In [6]:
# 定義爬取的遞迴深度。深度不要訂太深，否則會爬很久。
max_recursive_depth = 1

WikiArticle(root_keyword_link, input_keyword, 0)

遞迴層[0] - /wiki/%E6%8A%95%E8%B3%87 (投資)
遞迴層[1] - /wiki/%E6%8A%95%E6%9C%BA (投機)
遞迴層[1] - /wiki/%E8%B5%8C%E5%8D%9A (賭博)
遞迴層[1] - /wiki/%E6%95%88%E7%8E%87%E5%B8%82%E5%A0%B4%E5%81%87%E8%AA%AA (效率市場假說)
遞迴層[1] - /wiki/%E7%90%86%E6%80%A7 (理性)
遞迴層[1] - /wiki/%E8%82%A1%E6%9D%B1%E6%AC%8A%E7%9B%8A (股權)
遞迴層[1] - /wiki/%E5%82%A8%E8%93%84 (儲蓄)
遞迴層[1] - /wiki/%E7%BB%8F%E6%B5%8E%E5%AD%A6 (經濟學)
遞迴層[1] - /wiki/%E8%B5%84%E6%9C%AC (資本)
遞迴層[1] - /wiki/%E5%9B%BD%E5%86%85%E7%94%9F%E4%BA%A7%E6%80%BB%E5%80%BC (國內生產總值)
遞迴層[1] - /wiki/%E5%AD%98%E6%AC%BE%E4%BF%9D%E9%9A%AA (存款保險)
